In [13]:
import re
import sys
import glob
from datetime import datetime, date, timedelta

In [7]:
# Regex for the Apache common log format.
parts = [
    r'(?P<server>\S+)',                   # host %h
    r'(?P<host>\S+)',
    r'\S+',                             # indent %l (unused)
    r'(?P<user>\S+)',                   # user %u
    r'\[(?P<time>.+)\]',                # time %t
    r'"(?P<request>.*)"',               # request "%r"
    r'(?P<status>[0-9]+)',              # status %>s
    r'(?P<size>\S+)',                   # size %b (careful, can be '-')
    r'"(?P<referrer>.*)"',              # referrer "%{Referer}i"
    r'"(?P<agent>.*)"',                 # user agent "%{User-agent}i"
]
pattern = re.compile(r'\s+'.join(parts)+r'\s*\Z')

In [22]:
# Change Apache log items into Python types.
def pythonized(match):
    d = match.groupdict()
    
    # Clean up the request.
  
    # Some dashes become None.
    for k in ("user", "referrer", "agent"):
        if d[k] == "-":
            d[k] = None
      
    #  The size dash becomes 0.
    if d["size"] == "-":
        d["size"] = 0
    else:
        d["size"] = int(d["size"])
  
    # Convert the timestamp into a datetime object. Accept the server's time zone.
    # time, zone = d["time"].split()
    d["datetime"] = datetime.strptime(d['time'], "%d/%b/%Y:%H:%M:%S %z")
  
    return d

In [14]:
g = glob.glob('files/other_*')
g

['files/other_vhosts_access.log',
 'files/other_vhosts_access.log.1',
 'files/other_vhosts_access.log.2',
 'files/other_vhosts_access.log.2.gz',
 'files/other_vhosts_access.log.3',
 'files/other_vhosts_access.log.3.gz']

In [23]:
records = []
for file in g:
    if file.endswith('gz'):
        continue
    with open(file, 'r') as f:
        for line in f:
            records.append(pythonized(pattern.match(line)))

In [18]:
line

'recipe.jdstemmler.com:80 169.229.3.91 - - [29/Dec/2015:20:20:33 +0000] "\\x87\\x80\\xa9\\"\\x1c\\xcd\\xd34\\x86\\xe46\\x82x\\x7fV\\xdd\\x02\\xfb\\xe9\\xac\\xd3\\x87\\x7f\\xbd\\x8ag[\\x8a\\x838\\xb6$\\x01SE\\x0e\\x050@" 405 178 "-" "-"\n'

In [26]:
dd = pattern.match(line).groupdict()

In [27]:
dd

{'agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36',
 'host': '64.125.192.130',
 'referrer': '-',
 'request': 'GET / HTTP/1.1',
 'server': 'recipe.jdstemmler.com:80',
 'size': '808',
 'status': '500',
 'time': '21/Dec/2015:20:24:14 +0000',
 'user': '-'}

In [17]:
file

'files/other_vhosts_access.log.1'

In [28]:
df = pd.DataFrame(records)